In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [ ]:
import random

In [ ]:
import json
import copy

In [ ]:
import time
import datetime

In [ ]:
from collections import Counter

In [ ]:
from termcolor import colored

# Prepare the data structure 

In [ ]:
# Slot = Index , Day , Hour
class SlotProcessor:
    def __init__(self, inputString):
        stringParts = inputString.split(",")
        cursorPosition = 0
        self.id = stringParts[cursorPosition].strip()
        cursorPosition += 1
        self.day = stringParts[cursorPosition].strip()
        cursorPosition += 1
        self.time = stringParts[cursorPosition].strip()
    """        
    def getSlotInfo(self):
        print('Index: {} | Day: {} | Hour:{}'.format(self.id, self.day, self.time))
    """

In [ ]:
# Room = Index , Name , Capacity
class RoomProcessor:
    def __init__(self, inputString):
        stringParts = inputString.split(",")
        cursorPosition = 0

        self.id = stringParts[cursorPosition].strip()
        cursorPosition += 1

        self.name = stringParts[cursorPosition].strip()
    """
    def getRoomInfo(self):
        print('Index: {} | name: {}'.format(self.id, self.name))
    """

In [ ]:
#  Course = Index , Name , Capacity , Arrangement , InstructorIndex
class CourseProcessor:
    def __init__(self, inputString):
        stringParts = inputString.split(",")
        cursorPosition = 0

        self.id = stringParts[cursorPosition].strip()
        cursorPosition += 1

        self.name = stringParts[cursorPosition].strip()
        cursorPosition += 1

        self.capacity = stringParts[cursorPosition].strip()
        cursorPosition += 1

        self.arrangement = stringParts[cursorPosition].strip()
        cursorPosition += 1

        self.instructorIndex = int(stringParts[cursorPosition].strip())
   
    def getCourseInfo(self):
        print("Course ID: {} | Course Name: {} | Roomtype: {} | Arrangement: {} | InstructorIndex: {}"\
              .format(self.id, self.name, self.capacity, self.arrangement, self.instructorIndex))
        
    def getCourseName(self):
        return self.name
    def getCourseInstructorIndex(self):
        return int(self.instructorIndex)
     

In [ ]:
# Instructor = Index , Name 
class InstructorProcessor:
    def __init__(self, inputString):
        stringParts = inputString.split(",")
        cursorPosition = 0

        self.id = stringParts[cursorPosition].strip()
        cursorPosition += 1

        self.name = stringParts[cursorPosition].strip()
        cursorPosition += 1
     
    def getInstructorIndex(self):
        return int(self.id)
    
    def getInstructorName(self):
        return self.name
    
    def getInstructorInfo(self):
        print("Instructor ID: {} | Instructor Name: {}".format(self.id, self.name))
    
    # Highlight function getInstructorNameViaIndex(instructorIndex)
    def getInstructorNameViaIndex(self, instructorIndex):
        if int(self.id) == instructorIndex:
            print (self.name) 
        else:
            print('There is no instructor with this instructor index')


In [ ]:
slots = []
rooms = []
courses = []
instructors = []
with open('input4.txt') as fp:
    for line in fp:
        if line.startswith("#"):
        # ignore comments in input file
            continue
        lineSections = line.split("=")
        # print(lineSections)
        if lineSections[0].strip() == "Slot":
            #1 , one    ,  8:00
            # print(lineSections[1])
            slots.append(SlotProcessor(lineSections[1]))
            # slots
        elif lineSections[0].strip() == "Room":
            rooms.append(RoomProcessor(lineSections[1]))
        elif lineSections[0].strip() == "Course":
            courses.append(CourseProcessor(lineSections[1]))
        elif lineSections[0].strip() == "Instructor":
            instructors.append(InstructorProcessor(lineSections[1]))

#print('number of slots: {}, number of rooms: {}, number of courses: {}, number of instructors: {}'.\
#     format(len(slots), len(rooms), len(courses), len(instructors)))


In [ ]:
class courseInstructor:
    def __init__(self, an_item_in_courses, an_item_in_instructors):
        self.course = an_item_in_courses        
        self.instructor = an_item_in_instructors
    
    def printCourseInstructorInfo(self): # Don't forget situation that 2 teachers may have the same name.
        return('course name: {} | instructor name: {}'.\
              format(self.course.getCourseName(),\
                     self.instructor.getInstructorName())) 
    

# Class schedule 

In [ ]:
def InstructorNotAvailable(which_class, CurrentInstructor, ExistedSlots, which_slot):
    if which_class == 1:
        return False
    else:
        i = 1
        a = 0
        while i < which_class:          
            if CurrentInstructor == ExistedSlots[which_slot - 40 * i][0].instructor.getInstructorIndex():
                a += 1
            else:
                a += 0
            
            i += 1
        if a == 0:
            return False
        else:
            return True


In [ ]:
class Schedule:
    def __init__(self, slots, rooms, courses, instructors):
        self.schedule_slots = slots
        self.schedule_rooms = rooms
        self.schedule_courses = courses
        self.schedule_instructors = instructors
        self.slotsInfo = []
        self.slots = []
        self.fitness = 0.0
        self.generation = 1
        self.number_of_courses_for_a_class_per_week = len(self.schedule_slots)
        self.weekday = 5
        self.number_of_courses_for_a_class_per_day=self.number_of_courses_for_a_class_per_week/self.weekday

        for i in range(0, len(slots) * len(rooms)): # range(0, 240)
            self.slots.append([]) #Change self.slots from [] to [[],[],[]...]
            
    def inspectSchedule(self):
        if self.slots[0] == []:
                print('Wake up, before looking up the schedule, create it at first!')
        else:
                
            for i in range(0, len(slots) * len(rooms)):
                print( 'CourseInstructorInfo: {} | {}'\
                      .format(self.slots[i][0].printCourseInstructorInfo(), i))
    # only can be called once        
    def createSchedule(self):
        number_of_courses_for_all_classes_per_week = len(self.slots) # 240
        number_of_course_types = len(self.schedule_courses)//len(self.schedule_rooms) 
        initial_choice = [list(range(0,40)), list(range(40,80)),\
                          list(range(80,120)), list(range(120,160)),\
                          list(range(160,200)), list(range(200,240))]
        
        for course in self.schedule_courses:           
            slotContent = courseInstructor(course, self.schedule_instructors[course.getCourseInstructorIndex() - 1])
            classIndex=(int(course.id)+15)//number_of_course_types
            InstructorIndex = course.getCourseInstructorIndex() # type(classIndex) is float
            repeated_times_for_a_course = int(course.arrangement[1])
            j = 0 
            while j < repeated_times_for_a_course:
                if len(initial_choice[classIndex - 1]) > 1:
                    abondon_choice_1, abondon_choice_2 = random.sample(initial_choice[classIndex - 1], 2) 
                    abondon_choice = abondon_choice_2 if InstructorNotAvailable(classIndex, InstructorIndex, self.slots, abondon_choice_1)\
                    else abondon_choice_1
                else: 
                    abondon_choice = initial_choice[classIndex - 1][0] 
                        
                self.slots[abondon_choice].append(slotContent)
                """
                print('OKAY! The {}th course :{}, place it to the {}th slot'.format(counter_the_number_of_arranged_course,course.getCourseName(), abondon_choice))
                counter_the_number_of_arranged_course += 1
                print(self.slots[abondon_choice][0].printCourseInstructorInfo()) # Different from Version1
                """
                initial_choice[classIndex - 1].remove(abondon_choice)
                j = j + 1    
        
        for k in range(0, len(self.slots)):
            self.slotsInfo.append(self.slots[k][0])
          

# define Fitness calculation

In [ ]:
def calculateFitnessForAStrip(courseForADayAClass):
    getMark = 0
    className = courseForADayAClass[0]
    classArr= courseForADayAClass[1]
    #x = len(list(dict(Counter(courseForADayAClass[0])).values()))
    x = len(list(dict(Counter(className)).values()))
    
    if x==8:
        getMark += 10
        
    if '语文' or '数学' or '英语' in className[0:4]:
        getMark +=5
    if '数学' in className[0:4]:
        getMark == 5 
    if '英语' in className[0:4]:
        getMark == 5
    if '物理' in className[0:4]:
        getMark == 3
    if '化学' in className[0:4]:
        getMark == 3
    if '生物' in className[0:4]:
        getMark == 3
    return getMark

## Fitness for eight classes a day

**Evaluate a course order in a day**

Cut`ScheduleTable.slotsInfo`  into slices, A slices represents the courses for a class a day  

##  Fitness for final schedule of class

In [ ]:
def getCourseToBeEvaluated(ScheduleTable): # ScheduleTable.slotsInfo
    courseToBeEvaluated = []
    for i in range(0,30):
        courseToBeEvaluated.append([])
    i = 0
    for k in ScheduleTable.slotsInfo: # ScheduleTable.slotsInfo
        m =  i // 8        
        courseToBeEvaluated[m].append(k)
        i += 1
    return courseToBeEvaluated 

def getScheduleOfClass(courseToBeEvaluated):
    ScheduleOfClass = []
    x = len(courseToBeEvaluated)
    for i in range(x):
        ScheduleOfClass.append([[], []])
        
    for i in range(x):
        for j in courseToBeEvaluated[i]:
            ScheduleOfClass[i][0].append(j.course.name)
            ScheduleOfClass[i][1].append(j.course.arrangement)
    return ScheduleOfClass

def calculateFitness(ScheduleOfClass):
    mark = 0
    for ooo in ScheduleOfClass:
        a = calculateFitnessForAStrip(ooo)
        mark += a
    return mark 

In [ ]:
ScheduleTable = Schedule(slots, rooms, courses, instructors )
ScheduleTable.createSchedule()
courseToBeEvaluated = getCourseToBeEvaluated(ScheduleTable)
scheduleOfClass = getScheduleOfClass(courseToBeEvaluated)
calculateFitness(scheduleOfClass)

In [ ]:
scheduleOfClass

In [ ]:
scheduleOfClass[0][1]

In [ ]:
'A2' in scheduleOfClass[0][1]

In [ ]:
scheduleOfClass[0][0][4:8] 
Counter(scheduleOfClass[0][0][4:8])
dict(Counter(scheduleOfClass[0][0][4:8]))
dict(Counter(scheduleOfClass[0][0][4:8]))['音乐']

In [ ]:
scheduleOfClass[0][1][0:4]

In [ ]:
set(scheduleOfClass[0][1][0:4])

In [ ]:
#x = len(list(dict(Counter(courseForADayAClass[0])).values()))

In [ ]:
dict(Counter(scheduleOfClass[0][0])).values()

In [ ]:
set(scheduleOfClass[0][1][0:4]) == set(['A1', 'A2', 'A4', 'A5'])

In [ ]:
set([1, 3, 4]) == set([1, 4, 3]) == set([3, 4, 1]) == set([3,3, 4, 1])

In [ ]:
Counter([3, 3, 4, 1])

In [ ]:
set(Counter([3, 3, 4, 1]))

In [ ]:
len(set(Counter([3, 3, 4, 1])))

In [ ]:
Counter(['A1', 'A2', 'A4', 'A5']) == Counter(['A1', 'A2', 'A5', 'A4'])

Approaches of computing fitness
* Value class one by one 
* Value class order set
Distinguish the condition of perfect schedule of classes from hard condition.

In [ ]:
def calculateFitnessForAStrip(courseForADayAClass):
    getMark = 0
    className = courseForADayAClass[0]
    classArr= courseForADayAClass[1]
    #x = len(list(dict(Counter(courseForADayAClass[0])).values()))
    x = len(list(dict(Counter(className)).values()))
    
    
    if x==8:
        getMark += 20
    elif x==7:
        getMark += -20
    else:
        getMark += -50
        
    if '语文' in className[0:4]:
        getMark +=5
    else:
        getMark += -5
    if '数学' in className[0:4]:
        getMark == 5
    else:
        getMark += -5
    if '英语' in className[0:4]:
        getMark == 5
    else:
        getMark += -5
    if '物理' in className[0:4]:
        getMark += 3

    if '化学' in className[0:4]:
        getMark += 3

    if '生物' in className[0:4]:
        getMark += 3

    
    if 'A1' in classArr[4:8]:
        getMark += 5
    
    if 'A2' in classArr[4:8]:
        getMark += 5

       
            
    return getMark

In [ ]:
def calculateFitnessForAStrip(courseForADayAClass):
    getMark = 0
    className = courseForADayAClass[0]
    classArr= courseForADayAClass[1]
    nameCountDict = dict(Counter(className[0:4]))

    if '语文' in className[0:4] and nameCountDict['语文'] ==1:   
        getMark +=5
    if '数学' in className[0:4] and nameCountDict['数学'] ==1:
        getMark +=5   
    if '英语' in className[0:4] and nameCountDict['英语'] ==1:
        getMark +=5 

       
            
    return getMark

In [ ]:
"""
def calculateFitnessForAStrip(courseForADayAClass):
    getMark = 0
    classSet = courseToBeEvaluated[0]
    arrangementSet= courseForADayAClass[1]
    x = len(list(dict(Counter(courseForADayAClass[0])).values()))
    
    if Counter(arrangementSet[0:4]) == Counter(['A5','A5', 'A4', 'A3']) or Counter(['A5','A5', 'A3', 'A3']):
        getMark += 5
    
    elif Counter(arrangementSet[0:4]) == Counter(['A3','A3','A2','A1']) or Counter(['A3','A2','A2','A1']) or Counter(['A3','A2','A1','A1']):
        getMark += -5
    else:
        getMark += -1 
    if  Counter(arrangementSet[4:8]) == Counter(['A3','A3','A2','A1']) or Counter(['A3','A2','A2','A1']) or Counter(['A3','A2','A1','A1']):
        getMark += 5
    elif Counter(arrangementSet[4:8]) == Counter(['A5','A5', 'A4', 'A3']) or Counter(['A5','A5', 'A3', 'A3']):
        getMark += -5
    else:
        getMark += 0
    if x==8:
        getMark += 10
    elif x == 7:
        getMark = -2
    elif x == 6:
        getMark = -8
    else:
        getMark += -10
    return getMark
"""

## Test as a whole

In [ ]:
begin_time = time.time()
i = 0
fitness = []
while i < 100:
    ScheduleTable = Schedule(slots, rooms, courses, instructors )
    ScheduleTable.createSchedule()
    courseToBeEvaluated = getCourseToBeEvaluated(ScheduleTable)
    scheduleOfClass = getScheduleOfClass(courseToBeEvaluated)
    #calculateFitness(scheduleOfClass)
    fitness.append(calculateFitness(scheduleOfClass))
    i +=1 
cost_time = time.time() -begin_time

# Mutate a available schedule

In [ ]:
def validMutation(ScheduleTable):
    p4 = copy.deepcopy(ScheduleTable)
    t1 = random.randint(0, 5) 
    TimesOfInvalidMutation = 0

    classIndex1, classIndex2 = random.sample(range(40 * t1, 40 * t1 + 40), 2)
    while InstructorNotAvailable(t1+1, p4.slots[classIndex1][0].course.getCourseInstructorIndex(), p4.slots,  classIndex1) or \
          InstructorNotAvailable(t1+1, p4.slots[classIndex2][0].course.getCourseInstructorIndex(), p4.slots,  classIndex2):
        
        TimesOfInvalidMutation += 1
        classIndex1, classIndex2 = random.sample(range(40 * t1, 40 * t1 + 40), 2)
        print(colored("Times of Invalid mutaion: {}".format(TimesOfInvalidMutation),'red'))
        if TimesOfInvalidMutation > 20:
            # To make the return value same as input 
            p4.slotsInfo[classIndex1], p4.slotsInfo[classIndex2] = p4.slotsInfo[classIndex2], p4.slotsInfo[classIndex1] 
            ScheduleTable.generation = ScheduleTable.generation - 1
            print(colored('Prepare to stop the iteration','red'))
            break
   # print(colored('Mutation is valid now', 'green'))   
    p4.slotsInfo[classIndex1], p4.slotsInfo[classIndex2] = p4.slotsInfo[classIndex2], p4.slotsInfo[classIndex1]    
    p4.generation = ScheduleTable.generation + 1
    
    return p4

### Test `validMutation`

In [ ]:
ScheduleTable = Schedule(slots, rooms, courses, instructors )
ScheduleTable.createSchedule()
bestParent = ScheduleTable
courseToBeEvaluated = getCourseToBeEvaluated(bestParent)
scheduleOfClass = getScheduleOfClass(courseToBeEvaluated)
bestFitness = calculateFitness(scheduleOfClass)
x = 0
while True: 
    child = validMutation(bestParent) 
    #print(x)
    if child == bestParent:
        break
    courseToBeEvaluated = getCourseToBeEvaluated(child)
    scheduleOfClass = getScheduleOfClass(courseToBeEvaluated)
    childFitness = calculateFitness(scheduleOfClass)
    
    #print("parentFitness = {} | childFitness= {}".format(bestFitness, childFitness))
    if bestFitness >= childFitness:
        x += 1
        if x > 120:
            break
        continue  
    x = 0
    bestFitness = childFitness 
    #print("bestFitness = {}".format(bestFitness))
    bestParent = child  
bestParent.inspectSchedule()

After 3 days, I figured out, that it's not to find a valid mutation, it's find a child better than parent which cost most time.